In [1]:
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.functions import *

# Cell to create a spark session
spark = (
    SparkSession.builder.appName("MAST30034 ASSIGNMENT 1 DUSTIN")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.executor.memory", "2g")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

22/08/20 23:24:34 WARN Utils: Your hostname, DESKTOP-3ADPNV0 resolves to a loopback address: 127.0.1.1; using 172.25.23.95 instead (on interface eth0)
22/08/20 23:24:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/20 23:24:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
yellow = spark.read.parquet('../../mast30034-project-1-dustintano10/data/curated/yellow/yellow_credit')

In [3]:
yellow.groupBy('Date').count().orderBy('Date')


Date,count
2018-10-17,182129
2018-10-20,169804
2018-10-26,181843
2018-10-29,156363
2018-10-31,164490
2018-11-05,161472
2018-11-11,148113
2018-11-20,166765
2018-11-23,99489
2018-12-01,180716


In [4]:
# converts the pickup_hours into categorical data as there are correlation that could be seen from analysis

# drop the tpep pickup and dropoff columns

yellow = yellow.withColumn('pickup_hour', hour("tpep_pickup_datetime"))
yellow = yellow.withColumn('dropoff_hour', hour("tpep_dropoff_datetime"))

yellow = yellow.drop('tpep_pickup_datetime', 'tpep_dropoff_datetime')

In [5]:
# pick 2018-11-20 as train set after randomly picking it manually
yellow_train = yellow.where((F.col('Date') == '2018-11-20'))

# pick 2018-11-23 as the test set as it is the next game date after 2018-11-20
yellow_test = yellow.where((F.col('Date') == '2018-11-23'))



In [6]:
yellow_train = yellow_train.drop('Date','Start(ET)')

In [7]:
yellow_test = yellow_test.drop('Date', 'Start(ET)')